# This notebook shows system identification of Franka Emika Panda

In [1]:
from mujoco_logger import SimLog
from mujoco_sysid import mj_jointRegressor
import mujoco
from robot_descriptions.panda_mj_description import MJCF_PATH
from robot_descriptions.panda_description import URDF_PATH
import pinocchio as pin
from scripts.utils import ActuatorMotor, update_actuator
import numpy as np

In [2]:
log = SimLog("data/panda_data.json")

In [3]:
# cut part of the data to disable the gripper
q = log.data("qpos")[:, :-2]
v = log.data("qvel")[:, :-2]
dv = log.data("qacc")[:, :-2]
tau = log.data("ctrl")[:, :-1]

q.shape, v.shape, dv.shape, tau.shape

((5000, 7), (5000, 7), (5000, 7), (5000, 7))

In [8]:
model = mujoco.MjModel.from_xml_path(MJCF_PATH)
data = mujoco.MjData(model)

pinmodel = pin.buildModelFromUrdf(URDF_PATH)
pindata = pin.Data(pinmodel)

for actuator_id in range(model.nu):
    actuator = ActuatorMotor()
    update_actuator(model, actuator_id, actuator)

A = np.zeros((5000 * 7, 7 * 10))
b = np.zeros((5000 * 7,))

for i in range(len(log)):
    q_i = q[i]
    v_i = v[i]
    dv_i = dv[i]
    tau_i = tau[i]

    data.qpos[:-2] = q_i
    data.qvel[:-2] = v_i
    data.qacc[:-2] = dv_i

    mujoco.mj_inverse(model, data)
    mujoco.mj_rnePostConstraint(model, data)

    # remove fingers from consideration with [:-2, :-20]
    Ypin = pin.computeJointTorqueRegressor(
        pinmodel, pindata, log.qpin[i], log.vpin[i], log.dv[i]
    )[:-2, :-20]
    Y = mj_jointRegressor(model, data)[:-2, :-20]

    A[i * 7 : (i + 1) * 7, :] = Y
    b[i * 7 : (i + 1) * 7] = tau_i

    np.testing.assert_allclose(Y, Ypin, atol=1e-6, rtol=1e-6)

In [20]:
from mujoco import minimize

np.set_printoptions(precision=3, suppress=True)

for arr in np.hsplit(np.linalg.lstsq(A, b, rcond=None)[0], 7):
    print(arr)

[ 0.    -0.     0.     0.     0.    -0.     0.     0.    -0.     0.844]
[ 0.    -0.947 -2.421 -0.     0.593  0.31   0.252 -0.236  0.891 -2.707]
[ 0.554  1.89   0.184  1.085 -0.589  1.288 -1.526  0.696 -0.006 -0.223]
[ 0.698 -0.018  1.114 -0.077 -0.329 -0.15  -0.483  0.66   0.057  0.068]
[ 1.081 -0.047 -0.086  0.901  0.214  0.294 -0.475 -0.035 -0.162 -0.025]
[ 1.081 -0.935 -0.044  0.086 -0.043  0.038  0.232  0.181 -0.001  0.152]
[ 1.002 -0.265  0.062  0.047  0.045 -0.059  0.063  0.034  0.094  0.044]


In [13]:
for i in range(pinmodel.nbodies):
    print(pinmodel.names[i], pinmodel.inertias[i].toDynamicParameters())

universe [ 0.63  -0.026 -0.     0.031  0.005 -0.     0.007  0.001  0.     0.005]
panda_joint1 [ 4.971  0.019  0.01  -0.237  0.715 -0.     0.718  0.008  0.02   0.009]
panda_joint2 [ 0.647 -0.002 -0.019  0.002  0.009 -0.004  0.028  0.01   0.001  0.027]
panda_joint3 [ 3.229  0.089  0.127 -0.215  0.056 -0.008  0.053 -0.005 -0.004  0.018]
panda_joint4 [ 3.588 -0.191  0.375  0.099  0.068  0.028  0.032  0.004 -0.002  0.078]
panda_joint5 [ 1.226 -0.015  0.05  -0.047  0.039 -0.002  0.031 -0.005  0.002  0.011]
panda_joint6 [ 1.667  0.1   -0.024 -0.018  0.002  0.002  0.011 -0.     0.     0.012]
panda_joint7 [ 1.466  0.003  0.002  0.145  0.031  0.     0.028 -0.001 -0.001  0.007]
panda_finger_joint1 [0.015 0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
panda_finger_joint2 [0.015 0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
